In [2]:
import pandas as pd

# Load datasets
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

# Basic overview
print("Train shape:", train.shape)
print("Test shape:", test.shape)# Target column
target = 'Transport_Cost'

# Features = all columns except target
X = train.drop(columns=[target])
y = train[target]

# Test features
X_test = test.copy()

print("Feature shape:", X.shape)
print("Target shape:", y.shape)
print("Test feature shape:", X_test.shape)

train.head()


Train shape: (5000, 20)
Test shape: (500, 19)


,Hospital_Id,Supplier_Name,Supplier_Reliability,Equipment_Height,Equipment_Width,Equipment_Weight,Equipment_Type,Equipment_Value,Base_Transport_Fee,CrossBorder_Shipping,Urgent_Shipping,Installation_Service,Transport_Method,Fragile_Equipment,Hospital_Info,Rural_Hospital,Order_Placed_Date,Delivery_Date,Hospital_Location,Transport_Cost
0,fffe3200360030003700,Jo Valencia,0.44,21.0,6.0,NaN,NaN,3.62,17.13,No,No,No,Roadways,No,Working Class,No,10/20/17,10/20/17,APO AA 33776,179.50
1,fffe3400380037003400,Wanda Warren,0.58,29.0,20.0,1210684.0,Marble,9703.37,35.42,No,Yes,Yes,Roadways,No,Working Class,No,02/22/16,02/24/16,"South Kevin, VT 84493",627732.45
2,fffe3200350036003700,Robert Ackies,0.97,39.0,15.0,3305.0,Aluminium,40.21,18.54,No,No,No,Roadways,No,Working Class,No,01/11/18,01/10/18,"Kevinshire, NE 31279",1565.92
3,fffe3800320034003400,Charlotte Membreno,0.70,8.0,5.0,606.0,Brass,4.55,17.48,No,No,No,Roadways,No,Working Class,No,08/06/16,08/06/16,DPO AP 61572,257.71
4,fffe3600340033003000,Nena Silva,0.66,27.0,13.0,NaN,Marble,2726.80,30.23,Yes,No,No,Roadways,No,Working Class,NaN,12/15/16,12/17/16,"Joshuamouth, AK 01550",8553.52


In [3]:
train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Hospital_Id           5000 non-null   object 
 1   Supplier_Name         5000 non-null   object 
 2   Supplier_Reliability  4413 non-null   float64
 3   Equipment_Height      4717 non-null   float64
 4   Equipment_Width       4557 non-null   float64
 5   Equipment_Weight      4540 non-null   float64
 6   Equipment_Type        4401 non-null   object 
 7   Equipment_Value       5000 non-null   float64
 8   Base_Transport_Fee    5000 non-null   float64
 9   CrossBorder_Shipping  5000 non-null   object 
 10  Urgent_Shipping       5000 non-null   object 
 11  Installation_Service  5000 non-null   object 
 12  Transport_Method      3929 non-null   object 
 13  Fragile_Equipment     5000 non-null   object 
 14  Hospital_Info         5000 non-null   object 
 15  Rural_Hospital       

,Supplier_Reliability,Equipment_Height,Equipment_Width,Equipment_Weight,Equipment_Value,Base_Transport_Fee,Transport_Cost
count,4413.000000,4717.000000,4557.000000,4.540000e+03,5000.000000,5000.000000,5.000000e+03
mean,0.460925,21.759593,9.585034,4.097584e+05,1190.293363,37.374362,1.789806e+04
std,0.265964,11.944434,5.407542,2.808640e+06,8724.804213,26.907914,2.554261e+05
min,0.000000,3.000000,2.000000,3.000000e+00,3.000000,10.000000,-5.881832e+05
25%,0.230000,12.000000,6.000000,5.070000e+02,5.260000,16.630000,1.884150e+02
50%,0.450000,20.000000,8.000000,3.137500e+03,7.935000,23.410000,3.776050e+02
75%,0.670000,30.000000,12.000000,3.694150e+04,87.047500,57.805000,1.182702e+03
max,1.000000,73.000000,50.000000,1.179279e+08,382385.670000,99.980000,1.114343e+07


In [4]:
binary_cols = ['CrossBorder_Shipping', 'Urgent_Shipping', 'Installation_Service', 
               'Fragile_Equipment', 'Rural_Hospital']

for col in binary_cols:
    train[col] = train[col].map({'Yes': 1, 'No': 0})
    test[col] = test[col].map({'Yes': 1, 'No': 0})


In [5]:
train['Equipment_Weight'] = train['Equipment_Weight'].fillna(train['Equipment_Weight'].median())
test['Equipment_Weight'] = test['Equipment_Weight'].fillna(train['Equipment_Weight'].median())

train['Equipment_Type'] = train['Equipment_Type'].fillna('Unknown')
test['Equipment_Type'] = test['Equipment_Type'].fillna('Unknown')


In [6]:
train['Order_Placed_Date'] = pd.to_datetime(train['Order_Placed_Date'])
train['Delivery_Date'] = pd.to_datetime(train['Delivery_Date'])
test['Order_Placed_Date'] = pd.to_datetime(test['Order_Placed_Date'])
test['Delivery_Date'] = pd.to_datetime(test['Delivery_Date'])

train['Delivery_Days'] = (train['Delivery_Date'] - train['Order_Placed_Date']).dt.days
test['Delivery_Days'] = (test['Delivery_Date'] - test['Order_Placed_Date']).dt.days


C:\Users\Ayush Mishra\AppData\Local\Temp\ipykernel_23596\4060130828.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train['Order_Placed_Date'] = pd.to_datetime(train['Order_Placed_Date'])
C:\Users\Ayush Mishra\AppData\Local\Temp\ipykernel_23596\4060130828.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train['Delivery_Date'] = pd.to_datetime(train['Delivery_Date'])
C:\Users\Ayush Mishra\AppData\Local\Temp\ipykernel_23596\4060130828.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test['Order_Placed_Date'] = pd.to_datetime(test['Order_Placed_Date'])
C:\Users\Ayush

In [7]:
drop_cols = ['Hospital_Id', 'Order_Placed_Date', 'Delivery_Date']
train = train.drop(columns=drop_cols)
test = test.drop(columns=drop_cols)


In [8]:
cat_cols = ['Supplier_Name', 'Equipment_Type', 'Transport_Method', 'Hospital_Info', 'Hospital_Location']

train = pd.get_dummies(train, columns=cat_cols, drop_first=True)
test = pd.get_dummies(test, columns=cat_cols, drop_first=True)

test = test.reindex(columns=train.columns.drop('Transport_Cost'), fill_value=0)


In [9]:
# Target column
target = 'Transport_Cost'

# Features = all columns except target
X = train.drop(columns=[target])
y = train[target]

# Test features
X_test = test.copy()

print("Feature shape:", X.shape)
print("Target shape:", y.shape)
print("Test feature shape:", X_test.shape)


Feature shape: (5000, 9986)
Target shape: (5000,)
Test feature shape: (500, 9986)


In [13]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Step 1: Split train into training + validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Initialize XGBoost Regressor
xgb_model = XGBRegressor(
    n_estimators=100,       # Reduced for faster testing
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method='hist',
    n_jobs=-1
)

# Step 3: Train the model (no early stopping)
xgb_model.fit(X_train, y_train)

# Step 4: Predict on validation set
y_pred = xgb_model.predict(X_val)

# Step 5: Compute RMSE manually (compatible with all sklearn versions)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"Validation RMSE: {rmse:.2f}")


Validation RMSE: 34708.97


In [17]:
# Reload test.csv so Hospital_Id is available
import pandas as pd

test = pd.read_csv("data/test.csv")  # adjust path if needed
X_test = test.drop(columns=['Hospital_Id'])  # keep features only
# Predict transport cost for test set
test_predictions = xgb_model.predict(X_test)

# Prepare submission DataFrame
submission = pd.DataFrame({
    'Hospital_Id': test['Hospital_Id'],  # Use Hospital_Id from original test file
    'Transport_Cost': test_predictions
})

# Save to CSV
submission.to_csv("../outputs/submission.csv", index=False)
print("Submission file created!")


FileNotFoundError: [Errno 2] No such file or directory: 'data/test.csv'